## 2023/06/16

In [18]:
using IntervalArithmetic
using Random
using Distributions

In [52]:
include("./method1/index.jl")
include("./method2/index.jl")
include("./method3/index.jl")
include("./method4/index.jl")
include("./intervalPCM/index.jl")
include("./twofoldInterval/index.jl")
include("./twofoldIntervalPCM/index.jl")
include("./utils.jl")

print()

## PCM の生成法

$n$: 基準/代替案数

$N = \left\{ 1, 2, \dots, n \right\}$

$r_{\max}$: 区間 PCM $A_k$ の各成分の幅を定めるときのパラメータ

crisp PCM $A = \left( a_{ij} \right)_{n \times n}$ を与える.

$\mathrm{DM}_k$ の区間 PCM $A_k = \left( A_{kij} \right)_{n \times n} = \left( \left[ a_{kij}^\mathrm{L}, a_{kij}^\mathrm{U} \right] \right)_{n \times n}$ を次のように与える.

$$
\begin{align}
a_{kij}^\mathrm{L} & = a_{ij} \exp \left( -r_{kij}^\mathrm{L} \right), \quad
a_{kij}^\mathrm{U} & = a_{ij} \exp \left( r_{kij}^\mathrm{U} \right), \quad
i, j \in N, ~~ i < j.
\end{align}
$$
ただし, $r_{kij}^\mathrm{L}, r_{kij}^\mathrm{U} ~ (k \in \{ 1,2 \}, ~~ i, j \in N, ~ i < j)$ は $0$ から $r_{\max}$ の一様分布に従う独立同分布 (i.i.d.) である.

$i > j$ の成分は $A_{kij} = \left[ \frac{1}{a_{kij}^\mathrm{U}}, \frac{1}{a_{kij}^\mathrm{L}} \right]$ とする.

$a_{ij} \in \bigcap_k A_{kij}$ であること (すべての成分で共通部分が存在すること) が保証されている.

In [30]:
@inline function randamizedIntervalPCM(
    A::Matrix{T},
    λ::T,
    seed::Integer=1,
    )::Matrix{Interval{T}} where {T <: Real}
    Random.seed!(abs(seed)) # fix seed

    m, n = size(A)
    if m != n
        throw(ArgumentError("matrix A must be square."))
    end

    Aₖ = fill(1..1, (n, n))

    for i = 1:n
        for j = i:n
            if i == j continue end
            rₖᵢⱼᴸ = rand(Uniform(0, λ))
            rₖᵢⱼᵁ = rand(Uniform(0, λ))
            aₖᵢⱼᴸ = A[i,j] * exp(-rₖᵢⱼᴸ)
            aₖᵢⱼᵁ = A[i,j] * exp(rₖᵢⱼᵁ)
            Aₖ[i,j] = aₖᵢⱼᴸ..aₖᵢⱼᵁ
        end
    end

    for i = 1:n
        for j = 1:i
            if i == j continue end
            Aₖ[i,j] = 1/Aₖ[j,i]
        end
    end

    return Aₖ
end

function generateIPCMs(
    A::Matrix{T},
    λ::T,
    m::Integer, # Number of DMs
    length::Integer,
    seed::Integer=1
    )::Matrix{Matrix{Interval{T}}} where {T <: Real}
    Random.seed!(abs(seed)) # Fix seed to generate seeds to generate interval PCMs
    seeds = rand(UInt, (length, 2))
    return map(s -> randamizedIntervalPCM(A, λ, s), seeds)
end

generateIPCMs (generic function with 2 methods)

In [31]:
function runMethods(
    array_PCMs::Matrix{Matrix{Interval{T}}}
    )::Matrix{Matrix{TwofoldInterval{T}}} where {T <: Real}
    n, m = size(array_PCMs)
    
    results = Matrix{Matrix{TwofoldInterval{T}}}(undef, n, 4)

    for i in 1:n
        results[i, 1] = method1(array_PCMs[i, 1], array_PCMs[i, 2])
        results[i, 2] = method2(array_PCMs[i, 1], array_PCMs[i, 2])
        results[i, 3] = method3(array_PCMs[i, 1], array_PCMs[i, 2])
        results[i, 4] = method4(array_PCMs[i, 1], array_PCMs[i, 2])
    end

    return results
end

runMethods (generic function with 1 method)

In [55]:
A = [
    1 2 3 10
    1/2 1 3 8
    1/3 1/3 1 3
    1/10 1/8 1/3 1
]

len = 10
m = 2

array_Aₖ = generateIPCMs(A, 1.0, m, len, 1) # Generate k-th DM's interval PCM Aₖ
array_Ãₖ = generateRevisedMatrix.(array_Aₖ) # Generate k-th DM's twofold interval PCM Ãₖ
array_Ãₖ⁻ = map(Ãₖ -> Ãₖ[1], array_Ãₖ)
array_Ãₖ⁺ = map(Ãₖ -> Ãₖ[2], array_Ãₖ)

array_groupTwofoldIntervalPCMs = runMethods(array_Aₖ)

print()

<!-- ### 二つの区間の比較をするための関数

$A = \left[ a^\mathrm{L}, a^\mathrm{U} \right], B = \left[ b^\mathrm{L}, b^\mathrm{U} \right]$: 区間

1. $A \approx B$ の評価
    $$
    P \left( A, B \right) = \frac{\operatorname{Width}\left( A \cap B \right)}{\operatorname{Width}\left( A \cup B \right)}
    $$
2. $A \subseteq B$ の評価
    $$
    Q \left( A, B \right) = \frac{\operatorname{Width}\left( A \cap B \right)}{\operatorname{Width}(B)}
    $$ -->

### $DM_k$ の区間 PCM $A_k$ とグループの二重区間 PCM $\mathscr{A}$ の類似度の評価

1. $A_{kij} \approx \mathscr{A}_{ij}^-, A_{kij} \approx \mathscr{A}_{ij}^+$ の評価
    $$
    P\left( A_{kij}, \mathscr{A}_{ij}^- \right) = \frac{\operatorname{Width}\left( A_{kij} \cap \mathscr{A}_{ij}^- \right)}{\operatorname{Width}\left( A_{kij} \cup \mathscr{A}_{ij}^- \right)}, \quad
    P\left( A_{kij}, \mathscr{A}_{ij}^+ \right) = \frac{\operatorname{Width}\left( A_{kij} \cap \mathscr{A}_{ij}^+ \right)}{\operatorname{Width}\left( A_{kij} \cup \mathscr{A}_{ij}^+ \right)}, \quad
    i, j \in N, ~~ i \neq j.
    $$
2. $A_{kij} \supseteq \mathscr{A}_{ij}^-$ の評価
    $$
    R\left( A_{kij}, \mathscr{A}_{ij}^- \right) = \frac{\operatorname{Width}\left( A_{kij} \cap \mathscr{A}_{ij}^- \right)}{\operatorname{Width}\left( A_{kij} \right)}, \quad
    i, j \in N, ~~ i \neq j.
    $$
3. $A_{kij} \subseteq \mathscr{A}_{ij}^+$ の評価
    $$
    Q\left( A_{kij}, \mathscr{A}_{ij}^+ \right) = \frac{\operatorname{Width}\left( A_{kij} \cap \mathscr{A}_{ij}^+ \right)}{\operatorname{Width}\left( \mathscr{A}_{ij}^+ \right)}, \quad
    i, j \in N, ~~ i \neq j.
    $$

### $DM_k$ の二重区間 PCM $\tilde{A}_k$ とグループの二重区間 PCM $\mathscr{A}$ の類似度の評価

1. $\tilde{A}_{kij}^- \approx \mathscr{A}_{ij}^-, \tilde{A}_{kij}^+ \approx \mathscr{A}_{ij}^+$ の評価
    $$
    P\left( \tilde{A}_{kij}^-, \mathscr{A}_{ij}^- \right) = \frac{\operatorname{Width}\left( \tilde{A}_{kij}^- \cap \mathscr{A}_{ij}^- \right)}{\operatorname{Width}\left( \tilde{A}_{kij}^- \cup \mathscr{A}_{ij}^- \right)}, \quad
    P\left( \tilde{A}_{kij}^+, \mathscr{A}_{ij}^+ \right) = \frac{\operatorname{Width}\left( \tilde{A}_{kij}^+ \cap \mathscr{A}_{ij}^+ \right)}{\operatorname{Width}\left( \tilde{A}_{kij}^+ \cup \mathscr{A}_{ij}^+ \right)}, \quad
    i, j \in N, ~~ i \neq j.
    $$
2. $\tilde{A}_{kij}^- \subseteq \mathscr{A}_{ij}^-$ の評価  
    $\operatorname{Width}\left( \mathscr{A}_{ij}^- \right) \neq 0$ の場合,
    $$
    Q\left( \tilde{A}_{kij}^-, \mathscr{A}_{ij}^- \right) = \frac{\operatorname{Width}\left( A_{kij} \cap \mathscr{A}_{ij}^- \right)}{\operatorname{Width}\left( \mathscr{A}_{ij}^- \right)}, \quad
    i, j \in N, ~~ i \neq j.
    $$
3. $\tilde{A}_{kij}^+ \supseteq \mathscr{A}_{ij}^+$ の評価
    $$
    R\left( \tilde{A}_{kij}^+, \mathscr{A}_{ij}^+ \right) = \frac{\operatorname{Width}\left( A_{kij} \cap \mathscr{A}_{ij}^+ \right)}{\operatorname{Width}\left( \tilde{A}_{kij}^+ \right)}, \quad
    i, j \in N, ~~ i \neq j.
    $$

<!-- # means⁻ = [0, 0, 0, 0]

# for i = 1:len # i-th simulation
#     for j = 1:4 # j-th methods
#         means⁻[j] += meanOfNonDiagonalElements(P(array_Aₖ[i, 1], array_Ãₖ⁻[i, j]))
#         means⁻[j] += meanOfNonDiagonalElements(P(array_Aₖ[i, 2], array_Ãₖ⁻[i, j]))
#     end
# end

# means⁻ /= len * 2 -->